在无监督学习中，类似分类与回归的目标变量实现并不存在。无监督学习方法有聚类、关联分析算法等。

聚类将相似的对象归到同一个簇中，簇内对象越相似，聚类的效果越好。k均值（k-means）聚类之所以称为k-均值是因为它可以发现k个不同的簇，且每个簇的中心采用簇中所含值的均值计算而成。

簇识别（cluster identification）：簇识别给出聚类结果的含义。假设有一些数据，将相似数据归到一起，簇识别会告诉我们这些簇到底是些什么。

聚类与分类的不同：分类的目标事先已知，聚类则不一样。

### K-均值聚类算法
发现给定数据集的k个簇的算法。簇个数k是用户定的，每个簇通过其质心（centroid），即簇中所有点的中心去描述。

In [4]:
#一些辅助函数
from numpy import *

def loadDataSet(fileName):     
    dataMat = []              
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip().split('\t')
        fltLine = list(map(float,curLine)) 
        dataMat.append(fltLine)
    return dataMat

#计算两个向量的欧式距离
def distEclud(vecA, vecB):
    return sqrt(sum(power(vecA - vecB, 2))) 

#为dataSet构建一个包含k个随机质心的集合
def randCent(dataSet, k):
    n = shape(dataSet)[1]
    centroids = mat(zeros((k,n)))
    for j in range(n):#生成随机质心，确保随机点在数据边界内
        minJ = min(dataSet[:,j]) 
        rangeJ = float(max(dataSet[:,j]) - minJ)
        centroids[:,j] = mat(minJ + rangeJ * random.rand(k,1))
    return centroids

#测试以上函数的可用性
dataMat=mat(loadDataSet('testSet.txt'))
print(distEclud(dataMat[0],dataMat[1]))
randCent(dataMat,2)

5.184632816681332


matrix([[ 1.7067613 ,  4.87146034],
        [-0.14181446,  4.93160599]])

In [9]:
#伪代码
#创建k个点作为起始质点（经常是随机选择）
#当任意一个点的簇分配结果发生改变时
#    对数据集中的每个数据点
#        对每个质心
#            计算质心与数据点之间的距离
#        将数据点分配到距离最近的簇
#    对每一个簇，计算簇中所有点的均值并将均值作为质心

def kMeans(dataSet, k, distMeas=distEclud, createCent=randCent):#数据集、簇的数目、计算距离函数、创建初始质心函数
    m = shape(dataSet)[0]#数据点总数
    clusterAssment = mat(zeros((m,2)))#创建矩阵，存储每个电的簇分配结果 矩阵包含两列：簇索引值和误差  误差指当前点到簇质心的距离
    centroids = createCent(dataSet, k)
    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        for i in range(m):#为每个点寻找最近的质心
            minDist = inf; minIndex = -1
            for j in range(k):
                distJI = distMeas(centroids[j,:],dataSet[i,:])#计算距离
                if distJI < minDist:
                    minDist = distJI; minIndex = j
            if clusterAssment[i,0] != minIndex: clusterChanged = True
            clusterAssment[i,:] = minIndex,minDist**2
        print(centroids)#打印每次迭代结果
        for cent in range(k):#遍历质心
            ptsInClust = dataSet[nonzero(clusterAssment[:,0].A==cent)[0]]#获得给定簇的所有点
            centroids[cent,:] = mean(ptsInClust, axis=0) #计算所有点的均值
    return centroids, clusterAssment

myCentroids,clustAssing=kMeans(dataMat,4)

[[ 3.91236328  2.08430545]
 [-5.13333575  4.29716684]
 [-0.68447599  0.50174675]
 [-1.94171825  3.01732438]]
[[ 3.07100628  1.83263908]
 [-4.786473    3.358547  ]
 [-1.12814074 -2.83881026]
 [-2.14699026  2.86989111]]
[[ 2.95373358  2.32801413]
 [-4.12008     3.391434  ]
 [-0.83188333 -2.97988206]
 [-2.27726128  2.72025794]]
[[ 2.95373358  2.32801413]
 [-3.67036525  3.311776  ]
 [-0.83188333 -2.97988206]
 [-2.15933762  2.65627544]]
[[ 2.95373358  2.32801413]
 [-3.649876    2.955743  ]
 [-0.83188333 -2.97988206]
 [-2.0654322   2.73125307]]
[[ 2.95373358  2.32801413]
 [-3.48664243  2.77252571]
 [-0.83188333 -2.97988206]
 [-1.90956662  2.79537162]]
[[ 2.95373358  2.32801413]
 [-3.31050467  2.56852067]
 [-0.83188333 -2.97988206]
 [-1.76693827  2.96643864]]
[[ 2.95373358  2.32801413]
 [-3.2308203   2.5758848 ]
 [-0.83188333 -2.97988206]
 [-1.692266    2.9988663 ]]
[[ 2.95373358  2.32801413]
 [-3.17006745  2.60393509]
 [-0.83188333 -2.97988206]
 [-1.595569    3.01158056]]


可以看到给出了4个质心，算法在11次迭代后收敛。

下面将探讨如何评价聚类的质量：

一种用于度量聚类效果的指标是SSE（Sum of Squared Error，误差平方和），即上述代码中clusterAssment矩阵的第一列之和。SSE越小表示聚类效果越好，数据越接近质心。

那目标就是降低SSE，从而提升聚类效果。增加簇的个数可以降低SSE，但是这违背了聚类的目标。聚类的目标是在保持簇数目不变的情况下提高簇的质量。所以为了降低SSE,我们将具有最大SSE的簇划分为两个簇（办法：将最大簇包含的点过滤，在其上运行k-均值算法），同时为了保持簇总数不变，可以将某两个簇合并（办法：合并最近的质心或者合并两个使得SSE增幅最小的质心）。

In [12]:
#为了克服k-均值算法收敛于局部最小值的问题，提出二分k-均值算法（bisecting K-means）
#伪代码
#将所有点看出一个簇
#当簇数目小于k时
#    对于每一个簇
#        计算总误差
#        在给定的簇上面进行k-均值聚类（k=2）
#        计算将该簇一分为二之后的总误差
#    选择使得误差最小的那个簇进行划分操作

def biKmeans(dataSet, k, distMeas=distEclud):
    m = shape(dataSet)[0]
    #创建一个初始簇
    clusterAssment = mat(zeros((m,2)))#用于存储数据集中每个点的簇分配结果以及平方误差
    centroid0 = mean(dataSet, axis=0).tolist()[0]#计算整个数据集的质心
    centList =[centroid0] #使用列表保留所有的质心
    for j in range(m):
        clusterAssment[j,1] = distMeas(mat(centroid0), dataSet[j,:])**2
    while (len(centList) < k):
        lowestSSE = inf#开始将最小SSE设为无穷大
        for i in range(len(centList)):#遍历所有簇来寻找最佳的簇进行划分
            ptsInCurrCluster = dataSet[nonzero(clusterAssment[:,0].A==i)[0],:]#将簇中所有点看成一个小的数据集
            centroidMat, splitClustAss = kMeans(ptsInCurrCluster, 2, distMeas)#将该数据集合进行k=2的k-means，生成两个质心（簇）
            sseSplit = sum(splitClustAss[:,1])#给出簇的误差值
            sseNotSplit = sum(clusterAssment[nonzero(clusterAssment[:,0].A!=i)[0],1])
            print("sseSplit, and notSplit: ",sseSplit,sseNotSplit)
            if (sseSplit + sseNotSplit) < lowestSSE:#如果该划分SSE小，则本次划分被保留
                bestCentToSplit = i
                bestNewCents = centroidMat
                bestClustAss = splitClustAss.copy()
                lowestSSE = sseSplit + sseNotSplit
        #簇的划分，更新簇的分配结果，通过更改簇的编号，通过一下两个数组过滤器实现
        bestClustAss[nonzero(bestClustAss[:,0].A == 1)[0],0] = len(centList)
        bestClustAss[nonzero(bestClustAss[:,0].A == 0)[0],0] = bestCentToSplit
        print('the bestCentToSplit is: ',bestCentToSplit)
        print('the len of bestClustAss is: ', len(bestClustAss))
        centList[bestCentToSplit] = bestNewCents[0,:].tolist()[0]
        centList.append(bestNewCents[1,:].tolist()[0])#新的质心被添加到cenList
        clusterAssment[nonzero(clusterAssment[:,0].A == bestCentToSplit)[0],:]= bestClustAss
    return mat(centList), clusterAssment#返回质心列表与簇分配结果

datMat3=mat(loadDataSet('testSet2.txt'))
cenList,myNewAssments=biKmeans(datMat3,3)
print(cenList)

[[ 1.44986076 -2.381998  ]
 [ 4.29385369  2.61664937]]
[[-1.33989914 -1.10157321]
 [ 0.94818419  3.40211065]]
[[-0.97166321 -1.96674883]
 [ 0.38490422  3.35338275]]
[[-0.62407076 -2.58281924]
 [ 0.09338772  3.27587208]]
[[-0.45965615 -2.7782156 ]
 [-0.00675605  3.22710297]]
sseSplit, and notSplit:  453.0334895807502 0.0
the bestCentToSplit is:  0
the len of bestClustAss is:  60
[[-0.88316209 -2.76806119]
 [-0.92173948 -3.12148017]]
[[-0.7072145  -2.28554217]
 [-0.08831862 -3.51722575]]
[[-0.97628982 -2.21600445]
 [ 0.171785   -3.46536256]]
[[-1.12616164 -2.30193564]
 [ 0.35496167 -3.36033556]]
sseSplit, and notSplit:  12.753263136887313 423.8762401366249
[[-0.21035763  4.10515984]
 [ 0.6029864   3.38951945]]
[[-2.94737575  3.3263781 ]
 [ 2.93386365  3.12782785]]
sseSplit, and notSplit:  77.59224931775066 29.15724944412535
the bestCentToSplit is:  1
the len of bestClustAss is:  40
[[-0.45965615 -2.7782156 ]
 [-2.94737575  3.3263781 ]
 [ 2.93386365  3.12782785]]


### 总结
k-均值聚类优缺点：
* 优点：容易实现
* 缺点：可能收敛到局部最小值，在大规模数据集上收敛较慢
* 适用数据类型：数值型数据